In [1]:
import torch as t
import torch.nn as nn
import torch.optim as optim
import torchvision._utils as util
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
import pandas as pd
import cv2 as cv
import os
import sklearn.model_selection as sc
from torch.utils.data import TensorDataset, DataLoader

In [2]:
img = cv.imread("../Dataset/asl_alphabet_test/asl_alphabet_test/A_test.jpg")
img

array([[[255,   1,   1],
        [250,   0,   0],
        [247,   0,   2],
        ...,
        [241,   0,   5],
        [245,   1,   1],
        [249,   5,   0]],

       [[252,   1,   2],
        [237,   3,   5],
        [208,   7,  11],
        ...,
        [196,   7,  16],
        [209,   3,   2],
        [219,   5,   0]],

       [[241,   0,   1],
        [207,   8,  12],
        [186,  76,  82],
        ...,
        [189, 112, 120],
        [243, 112, 115],
        [170,  12,  12]],

       ...,

       [[240,   0,   4],
        [193,   5,  10],
        [144,  68,  69],
        ...,
        [144, 129, 133],
        [219, 128, 131],
        [140,  14,  13]],

       [[248,   0,   1],
        [222,  11,  14],
        [190,  54,  56],
        ...,
        [219, 129, 129],
        [252, 119, 116],
        [169,  16,   9]],

       [[252,   1,   2],
        [224,   2,   2],
        [173,   8,  11],
        ...,
        [138,  12,  11],
        [173,  17,  12],
        [179,  10,   2]]

In [3]:
x_train = []
y_train = []

folder_path = "/Users/bhara-zstch1566/CNN/Project/ResNet/Dataset/asl_alphabet_train/asl_alphabet_train"

for foldername in os.listdir(folder_path):
    
    inside = (folder_path+"/"+foldername)
    for filename in os.listdir(inside):
        img = cv.imread(inside+'/'+filename)
        x_train.append(img)
        y_train.append(foldername)



FileNotFoundError: [Errno 2] No such file or directory: '/Users/bhara-zstch1566/CNN/Project/ResNet/Dataset/asl_alphabet_train/asl_alphabet_train'

In [ ]:
label = np.unique(y_train).sort()

lab = sorted(list(set(y_train)),key=lambda x:x.upper())
mapping = {}
for ind,i in enumerate(lab):
    mapping[i]=ind+1


In [ ]:
for i in range(len(y_train)):
    y_train[i]=mapping[y_train[i]]

In [ ]:
len(y_train)

In [ ]:
dataset = {"image":[],"label":[]}

In [ ]:
for i in range(len(y_train)):
    dataset['image'].append(x_train[i])
    dataset["label"].append(y_train[i])


In [ ]:
dataset['image']

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),                   
    transforms.Grayscale(num_output_channels=1),  # convert to grayscale
    transforms.Resize((64, 64)),                  # resize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])   # normalize grayscale
])


def preprocess(example):
    example["image"] = [transform(img) for img in example["image"]]
    return example
dataset = preprocess(dataset)

In [ ]:
len(dataset['image'])

In [ ]:
dataset['image']

## Train Test Dev split

In [ ]:
x_train,x_dev,y_train,y_dev = sc.train_test_split(dataset['image'],dataset['label'],test_size=0.30,random_state=42,shuffle=True)

In [ ]:
y_train,y_dev = t.tensor(y_train),t.tensor(y_dev)

In [ ]:
x_train,x_dev = t.stack(x_train),t.stack(x_dev)

In [ ]:
x_train.shape

In [ ]:
x_dev.shape

In [ ]:
y_dev.shape

In [ ]:
y_train.shape

In [ ]:
x_test,x_dev,y_test,y_dev = sc.train_test_split(x_dev,y_dev,test_size=0.50,random_state=42,shuffle=True)

In [ ]:
print(x_train.shape,x_dev.shape,x_test.shape)

In [ ]:

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

dev_dataset = TensorDataset(x_dev, y_dev)
dev_loader = DataLoader(dev_dataset, batch_size=64, shuffle=False)

test_dataset = TensorDataset(x_test,y_test)
test_loader = DataLoader(test_dataset,batch_size=64, shuffle=False)

## Model   

In [ ]:
def conv(input_channels,output_channels):
    return  nn.Sequential(
        nn.Conv2d(input_channels,output_channels,3,1,1),
        nn.ReLU(),
        nn.Conv2d(output_channels,output_channels,3,1,1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16,self).__init__()
        self.layer = nn.Sequential(
            conv(1,64),
            conv(64,128),
            conv(128,256),
            conv(256,512),
            conv(512,512)
        )
        self.fc = nn.Sequential(
            nn.Linear(512*2*2, 4096),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 30)
        )

    def forward(self,x):
        x = self.layer(x)
        x = t.flatten(x, 1) 
        x = self.fc(x)
        return x

In [ ]:
model = VGG16()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)

In [ ]:
device = "mps" if t.mps.is_available() else "cpu"
device

In [ ]:
print(len(train_loader))

In [ ]:
epochs = 5
device = "mps" if t.mps.is_available() else "cpu"
model = model.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    cnt = 0
    for x, y in train_loader:         
        x, y = x.to(device), y.to(device)
        
        y_pred = model(x)
        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    correct = 0
    total = 0
    with t.no_grad():
        for x,y in dev_loader:

            x, y = x.to(device), y.to(device)
            
            outputs = model(x)
            _, predicted = t.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    
    print(f"Epoch [{epoch+1}/{5}], Loss: {running_loss / len(train_loader):.4f}, Dev Accuracy: {100*correct/total:.2f}%")

In [ ]:

correct = 0
total = 0
with t.no_grad():
    for x,y in test_loader:
        
        x, y = x.to(device), y.to(device)
        
        outputs = model(x)
        _, predicted = t.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")

In [ ]:
test_img = cv.imread("/Users/bhara-zstch1566/CNN/Project/ResNet/Dataset/asl_alphabet_test/asl_alphabet_test/A_test.jpg")
transform = transforms.Compose([
    transforms.ToPILImage(),                   
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((64, 64)),                 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])


test_img = transform(test_img)


In [ ]:
test_img
test_img = test_img.unsqueeze(0)  # shape: [1, 1, 64, 64]
device = "mps" if t.mps.is_available() else "cpu"
test_img = test_img.to(device)


In [ ]:
model.eval()
with t.no_grad():
    out = model(test_img)
    _, predicted = t.max(out.data, 1)

print(f"The predicted word in the picture is : '{lab[(predicted.item())-1]}'")


In [ ]:
folder_path = "/Users/bhara-zstch1566/CNN/Project/ResNet/Dataset/asl_alphabet_test/asl_alphabet_test"
crt = 0
tot = 0
for foldername in os.listdir(folder_path):
    
        img = cv.imread((folder_path+"/"+foldername))
        transform = transforms.Compose([
            transforms.ToPILImage(),                   
            transforms.Grayscale(num_output_channels=1),  
            transforms.Resize((64, 64)),                 
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])  
        ])
        test_img = transform(img)
        test_img = test_img.unsqueeze(0)  # shape: [1, 1, 64, 64]
        device = "mps" if t.mps.is_available() else "cpu"
        test_img = test_img.to(device)

        print()
        pred = foldername.split('_')
        with t.no_grad():
            out = model(test_img)
            _, predicted = t.max(out.data, 1)
        tot+=1
        crt += 1 if lab[(predicted.item())-1]==pred[0] else 0
        print(f"The predicted word in the picture is : '{lab[(predicted.item())-1]}' and actual word is : '{pred[0]}' output {lab[(predicted.item())-1]==pred[0]}")

print(f"\naccuracy : {crt/tot *100}")

In [ ]:
t.save(model.state_dict(), "model.pth")

In [ ]:
print(lab)